In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import winsound
miniconda_path = r"C:\Users\lisa.smith\Miniconda3"
download_path = r"C:\Users\lisa.smith\Downloads"


## The Functions

In [2]:
def create_unique_IDs(user_initials):
    global batch_ID, start_date, timestamp
    user = user_initials+'-'
    currentDateAndTime = dt.now()
    curr_dt = dt.today().strftime('%Y-%m-%d-')
    currentTime = currentDateAndTime.strftime("%H-%M")
    timestamp = currentDateAndTime.timestamp()-1660000000
    start_date= dt.today().strftime('%Y-%m-%d %H:%M:%S')
    batch_ID = user + curr_dt + currentTime
    return  batch_ID,start_date,timestamp

def create_frame(Data, tier=None):
    df = Data.copy()
    tier =str(tier)
    #Work out which column names to get rid of and drop them
    col_names = ['TIER1_MINcalc', 'TIER1_MAXcalc','TIER2_MINcalc','TIER2_MAXcalc','TIER3_MINcalc','TIER3_MAXcalc','TIER4_MINcalc','TIER4_MAXcalc','AdjTL1','AdjTL2','AdjTL3','AdjTL4']
    filtered_col_names = [name for name in col_names if tier not in name]  
    df2=df.copy()
    df2.drop(columns=filtered_col_names, inplace=True)
    
    #Change TL1_MAX for example to TL_MAX, same for min and adj...
    names = df2.columns.values.tolist()
    for name in names:
        if tier in name:
            new_name = name.replace(tier,'')
            df2 = df2.rename(columns={name:new_name})
            
    #Create Line_ID            
    df2['Line_ID'] = 'TL' + tier + '_'
    df2['Line_ID'] = df2['Line_ID'] + df2['SOURCE_CHARGE_ID'].astype(str)
    
    #Create Tier_Line       
    df2['TIER_LINE'] = tier
      
    
    #Get rid of rows for price categories that don't exist ie MIN is Nan.
    df2 = df2[~df2['TIER_MINcalc'].isna()]
    print(np.shape(df2.columns.values))
    return df2

In [19]:
create_unique_IDs('LS')
start_date

'2022-11-25 15:44:47'

## Part 1 Read File

In [20]:
%%time
MB_folderpath = r"S:\BSA SM Shared\11. Analytics & Reporting Team\11.20 The Move to Oracle\01.Process_Templates_and_data\17. How do I process Multiband\03 Alteryx_outputs"
input_filename = r"\multiband_add.xlsx"
data=pd.read_excel(MB_folderpath + input_filename)
data.head(3)

CPU times: total: 15.6 ms
Wall time: 498 ms


,PG_ID,Oracle_name,ITEM_NUMBER,ITEM_STATUS,CHANNEL,ITEM_CLASS,PRIMARY_UOM_CODE,VAT,B1_QTY,B1 EXVAT,...,TIER3_MINcalc,TIER3_MAXcalc,TIER4_MINcalc,TIER4_MAXcalc,AdjTL1,AdjTL2,AdjTL3,AdjTL4,Unnamed: 31,Unnamed: 32
0,6795,6795 T03 Ncf CoreCat Split Awc Npm 6014,CVW139,Transacted,e-Direct,Category_Tower_11,PAC,20,1,23.917500,...,NaN,NaN,NaN,NaN,12.680,NaN,NaN,NaN,6795 T03 Ncf CoreCat Split Awc Npm 6014_CVW139,NaN
1,5568,ICF NCF Band 1 0 24 9,FAM22905,Transacted,e-Direct,Category_Tower_11,EA,20,1,256.755833,...,NaN,NaN,NaN,NaN,10.271,20.542,NaN,NaN,ICF NCF Band 1 0 24 9_FAM22905,NaN
2,5568,ICF NCF Band 1 0 24 9,FCB19097,Transacted,e-Direct,Category_Tower_02,EA,20,1,7.507500,...,19.0,34.0,34.0,NaN,0.074,0.149,0.224,0.298,ICF NCF Band 1 0 24 9_FCB19097,NaN


## Remove Duplicate Pricelists and Create Unique ID

In [21]:
pricelists_lookup = data[['Oracle_name',]]
pricelist_unique=pricelists_lookup.drop_duplicates()
pricelist_unique['PRICELIST_ID'] = np.arange(len(pricelist_unique))+round(timestamp*100)
pricelist_unique.head(3)               

C:\Users\lisa.smith\AppData\Local\Temp\ipykernel_6396\3589316032.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pricelist_unique['PRICELIST_ID'] = np.arange(len(pricelist_unique))+round(timestamp*100)


,Oracle_name,PRICELIST_ID
0,6795 T03 Ncf CoreCat Split Awc Npm 6014,939108746
1,ICF NCF Band 1 0 24 9,939108747
13,Northern Ireland Core Catalogue,939108748


## Create Unique IDs for Items and Charge and static parameters

In [22]:
# TODO - tidy first part of this code to be the same syntax as 2nd part
data['BATCH_NAME'] = batch_ID
data['SOURCE_PRICELIST_ITEM_ID'] = np.arange(len(data))+round(timestamp*1000)
data['SOURCE_CHARGE_ID'] = np.arange(len(data))+round(timestamp*10000)
data['NEW_SOURCE_CHARGE_ID'] = ''
data['OP_CODE_NO_OP'] = 'NO-OP'
data['OP_CODE_CREATE'] = 'CREATE'
data['START_DATE'] = start_date
data['SOURCE_TIERED_PRICING_ID']='TH_'+data['SOURCE_CHARGE_ID'].astype(str)
data['ITEM_LEVEL_CODE'] = 'ITEM'
data['PRIMARY_PRICING_UOM'] ='Y'
data = data.assign(emptycol1= '',emptycol2= '',emptycol3= '',emptycol4= '',emptycol5= '',
emptycol6= '',emptycol7= '',emptycol8= '', CAN_ADJUST_FLAG='Y',ENFORCE_ADJ_CALCULATION_FLAG='Y',
LINE_TYPE_CODE='ORA_BUY',PARENT_ENTITY_TYPE_CODE='PRICE_LIST_ITEM',
CHARGE_LINE_NUMBER=1, CHARGE_DEFINITION_CODE= 'QP_SALE_PRICE',CALCULATION_METHOD_CODE='PRICE',
AGGREGATION_METHOD_CODE='ORA_ON_LINE',TIER_BASIS_TYPE_CODE='ORA_ITEM_QUANTITY',
GRADUATED_CODE='HIGHEST_TIER',APPLICATION_METHOD_CODE='PER_UNIT',ADJUSTMENT_TYPE_CODE='DISCOUNT_AMOUNT',
ADJUSTMENT_BASIS_NAME='BASE_PRICE')
 
data.head(3)     


,PG_ID,Oracle_name,ITEM_NUMBER,ITEM_STATUS,CHANNEL,ITEM_CLASS,PRIMARY_UOM_CODE,VAT,B1_QTY,B1 EXVAT,...,PARENT_ENTITY_TYPE_CODE,CHARGE_LINE_NUMBER,CHARGE_DEFINITION_CODE,CALCULATION_METHOD_CODE,AGGREGATION_METHOD_CODE,TIER_BASIS_TYPE_CODE,GRADUATED_CODE,APPLICATION_METHOD_CODE,ADJUSTMENT_TYPE_CODE,ADJUSTMENT_BASIS_NAME
0,6795,6795 T03 Ncf CoreCat Split Awc Npm 6014,CVW139,Transacted,e-Direct,Category_Tower_11,PAC,20,1,23.917500,...,PRICE_LIST_ITEM,1,QP_SALE_PRICE,PRICE,ORA_ON_LINE,ORA_ITEM_QUANTITY,HIGHEST_TIER,PER_UNIT,DISCOUNT_AMOUNT,BASE_PRICE
1,5568,ICF NCF Band 1 0 24 9,FAM22905,Transacted,e-Direct,Category_Tower_11,EA,20,1,256.755833,...,PRICE_LIST_ITEM,1,QP_SALE_PRICE,PRICE,ORA_ON_LINE,ORA_ITEM_QUANTITY,HIGHEST_TIER,PER_UNIT,DISCOUNT_AMOUNT,BASE_PRICE
2,5568,ICF NCF Band 1 0 24 9,FCB19097,Transacted,e-Direct,Category_Tower_02,EA,20,1,7.507500,...,PRICE_LIST_ITEM,1,QP_SALE_PRICE,PRICE,ORA_ON_LINE,ORA_ITEM_QUANTITY,HIGHEST_TIER,PER_UNIT,DISCOUNT_AMOUNT,BASE_PRICE


## Join column with Pricelist IDs

In [23]:
data_joined = pd.merge(data,pricelist_unique,on='Oracle_name',how='inner')
data_joined.head(3)
print(np.shape(data_joined))

(30, 65)


In [24]:
columns_list=list(data_joined.columns)
#print(columns_list)
columns_list

['PG_ID',
 'Oracle_name',
 'ITEM_NUMBER',
 'ITEM_STATUS',
 'CHANNEL',
 'ITEM_CLASS',
 'PRIMARY_UOM_CODE',
 'VAT',
 'B1_QTY',
 'B1 EXVAT',
 'B2_QTY',
 'B2 EXVAT',
 'B3_QTY',
 'B3 EXVAT',
 'B4_QTY',
 'B4 EXVAT',
 'B5_QTY',
 'B5 EXVAT',
 'Num_tiers',
 'TIER1_MINcalc',
 'TIER1_MAXcalc',
 'TIER2_MINcalc',
 'TIER2_MAXcalc',
 'TIER3_MINcalc',
 'TIER3_MAXcalc',
 'TIER4_MINcalc',
 'TIER4_MAXcalc',
 'AdjTL1',
 'AdjTL2',
 'AdjTL3',
 'AdjTL4',
 'Unnamed: 31',
 'Unnamed: 32',
 'BATCH_NAME',
 'SOURCE_PRICELIST_ITEM_ID',
 'SOURCE_CHARGE_ID',
 'NEW_SOURCE_CHARGE_ID',
 'OP_CODE_NO_OP',
 'OP_CODE_CREATE',
 'START_DATE',
 'SOURCE_TIERED_PRICING_ID',
 'ITEM_LEVEL_CODE',
 'PRIMARY_PRICING_UOM',
 'emptycol1',
 'emptycol2',
 'emptycol3',
 'emptycol4',
 'emptycol5',
 'emptycol6',
 'emptycol7',
 'emptycol8',
 'CAN_ADJUST_FLAG',
 'ENFORCE_ADJ_CALCULATION_FLAG',
 'LINE_TYPE_CODE',
 'PARENT_ENTITY_TYPE_CODE',
 'CHARGE_LINE_NUMBER',
 'CHARGE_DEFINITION_CODE',
 'CALCULATION_METHOD_CODE',
 'AGGREGATION_METHOD_CODE',

## Create CSV sheets

In [25]:
Pricelists_int = data_joined[['BATCH_NAME','OP_CODE_NO_OP','PRICELIST_ID','Oracle_name',]]
Pricelists_int_unique=Pricelists_int.drop_duplicates(subset=['Oracle_name'])
Pricelists_int_unique.head(3)

,BATCH_NAME,OP_CODE_NO_OP,PRICELIST_ID,Oracle_name
0,LS-2022-11-25-15-44,NO-OP,939108746,6795 T03 Ncf CoreCat Split Awc Npm 6014
7,LS-2022-11-25-15-44,NO-OP,939108747,ICF NCF Band 1 0 24 9
19,LS-2022-11-25-15-44,NO-OP,939108748,Northern Ireland Core Catalogue


In [26]:
PriceListItems_int=data_joined[['OP_CODE_CREATE','PRICELIST_ID','SOURCE_PRICELIST_ITEM_ID','ITEM_LEVEL_CODE',
'ITEM_NUMBER','emptycol1','emptycol2','PRIMARY_UOM_CODE','PRIMARY_PRICING_UOM','LINE_TYPE_CODE']]
PriceListItems_int.head(3)

,OP_CODE_CREATE,PRICELIST_ID,SOURCE_PRICELIST_ITEM_ID,ITEM_LEVEL_CODE,ITEM_NUMBER,emptycol1,emptycol2,PRIMARY_UOM_CODE,PRIMARY_PRICING_UOM,LINE_TYPE_CODE
0,CREATE,939108746,9391087456,ITEM,CVW139,,,PAC,Y,ORA_BUY
1,CREATE,939108746,9391087464,ITEM,FDD1989,,,EA,Y,ORA_BUY
2,CREATE,939108746,9391087465,ITEM,FDD2413,,,EA,Y,ORA_BUY


In [27]:
PriceListCharges_int=data_joined[['OP_CODE_CREATE','PRICELIST_ID','SOURCE_CHARGE_ID','NEW_SOURCE_CHARGE_ID',
'PARENT_ENTITY_TYPE_CODE','SOURCE_PRICELIST_ITEM_ID','CHARGE_LINE_NUMBER','CHARGE_DEFINITION_CODE','emptycol1',
'emptycol2', 'CALCULATION_METHOD_CODE','B1 EXVAT','emptycol3', 'emptycol4',
 'emptycol5','emptycol6','emptycol7','CAN_ADJUST_FLAG','START_DATE']]
PriceListCharges_int.head(3)

,OP_CODE_CREATE,PRICELIST_ID,SOURCE_CHARGE_ID,NEW_SOURCE_CHARGE_ID,PARENT_ENTITY_TYPE_CODE,SOURCE_PRICELIST_ITEM_ID,CHARGE_LINE_NUMBER,CHARGE_DEFINITION_CODE,emptycol1,emptycol2,CALCULATION_METHOD_CODE,B1 EXVAT,emptycol3,emptycol4,emptycol5,emptycol6,emptycol7,CAN_ADJUST_FLAG,START_DATE
0,CREATE,939108746,93910874560,,PRICE_LIST_ITEM,9391087456,1,QP_SALE_PRICE,,,PRICE,23.917500,,,,,,Y,2022-11-25 15:44:47
1,CREATE,939108746,93910874568,,PRICE_LIST_ITEM,9391087464,1,QP_SALE_PRICE,,,PRICE,66.300833,,,,,,Y,2022-11-25 15:44:47
2,CREATE,939108746,93910874569,,PRICE_LIST_ITEM,9391087465,1,QP_SALE_PRICE,,,PRICE,16.700000,,,,,,Y,2022-11-25 15:44:47


In [28]:
TIER_HEADERS_int=data_joined[['OP_CODE_CREATE','PRICELIST_ID','SOURCE_CHARGE_ID','SOURCE_TIERED_PRICING_ID',
'AGGREGATION_METHOD_CODE','TIER_BASIS_TYPE_CODE','emptycol1','emptycol2','GRADUATED_CODE','APPLICATION_METHOD_CODE',
'ADJUSTMENT_TYPE_CODE','emptycol3','emptycol4','emptycol5','ENFORCE_ADJ_CALCULATION_FLAG']]
TIER_HEADERS_int

,OP_CODE_CREATE,PRICELIST_ID,SOURCE_CHARGE_ID,SOURCE_TIERED_PRICING_ID,AGGREGATION_METHOD_CODE,TIER_BASIS_TYPE_CODE,emptycol1,emptycol2,GRADUATED_CODE,APPLICATION_METHOD_CODE,ADJUSTMENT_TYPE_CODE,emptycol3,emptycol4,emptycol5,ENFORCE_ADJ_CALCULATION_FLAG
0,CREATE,939108746,93910874560,TH_93910874560,ORA_ON_LINE,ORA_ITEM_QUANTITY,,,HIGHEST_TIER,PER_UNIT,DISCOUNT_AMOUNT,,,,Y
1,CREATE,939108746,93910874568,TH_93910874568,ORA_ON_LINE,ORA_ITEM_QUANTITY,,,HIGHEST_TIER,PER_UNIT,DISCOUNT_AMOUNT,,,,Y
2,CREATE,939108746,93910874569,TH_93910874569,ORA_ON_LINE,ORA_ITEM_QUANTITY,,,HIGHEST_TIER,PER_UNIT,DISCOUNT_AMOUNT,,,,Y
3,CREATE,939108746,93910874570,TH_93910874570,ORA_ON_LINE,ORA_ITEM_QUANTITY,,,HIGHEST_TIER,PER_UNIT,DISCOUNT_AMOUNT,,,,Y
4,CREATE,939108746,93910874571,TH_93910874571,ORA_ON_LINE,ORA_ITEM_QUANTITY,,,HIGHEST_TIER,PER_UNIT,DISCOUNT_AMOUNT,,,,Y
5,CREATE,939108746,93910874572,TH_93910874572,ORA_ON_LINE,ORA_ITEM_QUANTITY,,,HIGHEST_TIER,PER_UNIT,DISCOUNT_AMOUNT,,,,Y
6,CREATE,939108746,93910874581,TH_93910874581,ORA_ON_LINE,ORA_ITEM_QUANTITY,,,HIGHEST_TIER,PER_UNIT,DISCOUNT_AMOUNT,,,,Y
7,CREATE,939108747,93910874561,TH_93910874561,ORA_ON_LINE,ORA_ITEM_QUANTITY,,,HIGHEST_TIER,PER_UNIT,DISCOUNT_AMOUNT,,,,Y
8,CREATE,939108747,93910874562,TH_93910874562,ORA_ON_LINE,ORA_ITEM_QUANTITY,,,HIGHEST_TIER,PER_UNIT,DISCOUNT_AMOUNT,,,,Y
9,CREATE,939108747,93910874563,TH_93910874563,ORA_ON_LINE,ORA_ITEM_QUANTITY,,,HIGHEST_TIER,PER_UNIT,DISCOUNT_AMOUNT,,,,Y


In [29]:
Data1 = create_frame(data_joined.copy(),tier=1)
Data2 = create_frame(data_joined.copy(),tier=2)
Data3 = create_frame(data_joined.copy(),tier=3)
Data4 = create_frame(data_joined.copy(),tier=4)


Tierlines_Data = pd.concat([Data1,Data2,Data3,Data4]).sort_values(['SOURCE_CHARGE_ID','TIER_LINE',])
Tierlines_Data




(58,)
(58,)
(58,)
(58,)


,PG_ID,Oracle_name,ITEM_NUMBER,ITEM_STATUS,CHANNEL,ITEM_CLASS,PRIMARY_UOM_CODE,VAT,B_QTY,B EXVAT,...,ADJUSTMENT_BASIS_NAME,PRICELIST_ID,Line_ID,TIER_LINE,B1_QTY,B1 EXVAT,Unnamed: 31,emptycol1,Unnamed: 1,Unnamed: 2
0,6795,6795 T03 Ncf CoreCat Split Awc Npm 6014,CVW139,Transacted,e-Direct,Category_Tower_11,PAC,20,1,23.917500,...,BASE_PRICE,939108746,TL1_93910874560,1,NaN,NaN,NaN,NaN,NaN,NaN
7,5568,ICF NCF Band 1 0 24 9,FAM22905,Transacted,e-Direct,Category_Tower_11,EA,20,1,256.755833,...,BASE_PRICE,939108747,TL1_93910874561,1,NaN,NaN,NaN,NaN,NaN,NaN
7,5568,ICF NCF Band 1 0 24 9,FAM22905,Transacted,e-Direct,Category_Tower_11,EA,20,2,246.484917,...,BASE_PRICE,939108747,TL2_93910874561,2,1.0,256.755833,ICF NCF Band 1 0 24 9_FAM22905,,NaN,NaN
8,5568,ICF NCF Band 1 0 24 9,FCB19097,Transacted,e-Direct,Category_Tower_02,EA,20,1,7.507500,...,BASE_PRICE,939108747,TL1_93910874562,1,NaN,NaN,NaN,NaN,NaN,NaN
8,5568,ICF NCF Band 1 0 24 9,FCB19097,Transacted,e-Direct,Category_Tower_02,EA,20,5,7.433917,...,BASE_PRICE,939108747,TL2_93910874562,2,1.0,7.507500,ICF NCF Band 1 0 24 9_FCB19097,,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,1786,Northern Ireland Core Catalogue,TLC85010,Transacted,e-Direct,Category_Tower_05,EA,20,101,120.618583,...,BASE_PRICE,939108748,TL4_93910874588,4,1.0,126.804167,Northern Ireland Core Catalogue_TLC85010,,NaN,NaN
29,1,National,TLP85000,Transacted,e-Direct,Category_Tower_05,EA,20,1,219.000000,...,BASE_PRICE,939108751,TL1_93910874589,1,NaN,NaN,NaN,NaN,NaN,NaN
29,1,National,TLP85000,Transacted,e-Direct,Category_Tower_05,EA,20,2,216.810000,...,BASE_PRICE,939108751,TL2_93910874589,2,1.0,219.000000,National_TLP85000,,NaN,NaN
29,1,National,TLP85000,Transacted,e-Direct,Category_Tower_05,EA,20,51,212.430000,...,BASE_PRICE,939108751,TL3_93910874589,3,1.0,219.000000,NaN,,National_TLP85000,NaN


In [30]:
TIER_LINES_int = Tierlines_Data[['OP_CODE_CREATE','PRICELIST_ID','SOURCE_TIERED_PRICING_ID','Line_ID','TIER_LINE','TIER_MINcalc','TIER_MAXcalc','APPLICATION_METHOD_CODE','ADJUSTMENT_TYPE_CODE','AdjTL','ADJUSTMENT_BASIS_NAME']]
TIER_LINES_int

,OP_CODE_CREATE,PRICELIST_ID,SOURCE_TIERED_PRICING_ID,Line_ID,TIER_LINE,TIER_MINcalc,TIER_MAXcalc,APPLICATION_METHOD_CODE,ADJUSTMENT_TYPE_CODE,AdjTL,ADJUSTMENT_BASIS_NAME
0,CREATE,939108746,TH_93910874560,TL1_93910874560,1,4.0,NaN,PER_UNIT,DISCOUNT_AMOUNT,12.680,BASE_PRICE
7,CREATE,939108747,TH_93910874561,TL1_93910874561,1,1.0,5.0,PER_UNIT,DISCOUNT_AMOUNT,10.271,BASE_PRICE
7,CREATE,939108747,TH_93910874561,TL2_93910874561,2,5.0,NaN,PER_UNIT,DISCOUNT_AMOUNT,20.542,BASE_PRICE
8,CREATE,939108747,TH_93910874562,TL1_93910874562,1,4.0,9.0,PER_UNIT,DISCOUNT_AMOUNT,0.074,BASE_PRICE
8,CREATE,939108747,TH_93910874562,TL2_93910874562,2,9.0,19.0,PER_UNIT,DISCOUNT_AMOUNT,0.149,BASE_PRICE
...,...,...,...,...,...,...,...,...,...,...,...
22,CREATE,939108748,TH_93910874588,TL4_93910874588,4,150.0,NaN,PER_UNIT,DISCOUNT_AMOUNT,10.309,BASE_PRICE
29,CREATE,939108751,TH_93910874589,TL1_93910874589,1,1.0,50.0,PER_UNIT,DISCOUNT_AMOUNT,2.190,BASE_PRICE
29,CREATE,939108751,TH_93910874589,TL2_93910874589,2,50.0,100.0,PER_UNIT,DISCOUNT_AMOUNT,6.570,BASE_PRICE
29,CREATE,939108751,TH_93910874589,TL3_93910874589,3,100.0,150.0,PER_UNIT,DISCOUNT_AMOUNT,10.950,BASE_PRICE


## Output CSVs to Downloads Folder

In [31]:
headers_output_file = r'\PriceListsHeaders.csv'
Pricelists_int_unique.to_csv(download_path + headers_output_file,header=False,index=False)
items_output_file = r'\PriceListsItems.csv'
PriceListItems_int.to_csv(download_path + items_output_file,header=False,index=False)
charges_output_file = r'\PriceListsCharges.csv'
PriceListCharges_int.to_csv(download_path + charges_output_file,header=False,index=False)
tierheaders_output_file = r'\TierHeadersInterface.csv'
TIER_HEADERS_int.to_csv(download_path + tierheaders_output_file,header=False,index=False)
tierlines_output_file = r'\TierLinesInterface.csv'
TIER_LINES_int.to_csv(download_path + tierlines_output_file,header=False,index=False)

## Create audit log

In [32]:
auditlog=data_joined[['BATCH_NAME','PRICELIST_ID','Oracle_name',
'SOURCE_PRICELIST_ITEM_ID','ITEM_NUMBER','SOURCE_CHARGE_ID','NEW_SOURCE_CHARGE_ID',
'PRIMARY_UOM_CODE','CHARGE_LINE_NUMBER','B1 EXVAT','START_DATE']]
auditlog.head(3)
date= dt.today().strftime('%Y-%m-%d')
auditlog_filepath=MB_folderpath+r'\03 Auditlog'
audit_log_filename=r'\Auditlog_MB_add'+ date+'.xlsx'
auditlog.to_excel(download_path +audit_log_filename,index=False)